<a href="https://colab.research.google.com/github/anjli01/PySpark-Notes/blob/main/Interview_Question_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Interview Question Generator


# Install the Google Generative AI library if you haven't already
# !pip install -q -U google-generativeai

import google.generativeai as genai
import os
import ipywidgets as widgets
from IPython.display import Markdown, display, clear_output
import json
import time





GEMINI_EMBEDDING = "gemini-embedding-001"
GEMINI_FLASH_LITE = "gemini-2.5-flash-lite-preview-09-2025"
GEMINI_3_FLASH = "gemini-3-flash-preview"
GEMINI_3_PRO = "gemini-3-pro-preview"

os.environ["GOOGLE_API_KEY"] = ""    # YOUR_GEMINI_API_KEY
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))





prompt_dictionary = {
    "MCQ (Single Correct)": """
Generate {number_of_questions} multiple-choice questions for a {role} interview.
Level: {level}
Topic(s): '''{topic}'''

Output MUST be a raw JSON list of dictionaries with this exact structure:
[
    {{
        "question": "The question text",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct_index": 0,
        "explanation": "A short explanation of why the answer is correct."
    }}
]

Important:
- Return ONLY the JSON code block.
- Ensure correct_index is an integer representing the position in the options list.
""",
    "MCQ (Multiple Correct)": """
Generate {number_of_questions} 'Multiple Correct' choice questions for a {role} interview.
Level: {level}
Topic(s): '''{topic}'''

Output MUST be a raw JSON list of dictionaries with this exact structure:
[
    {{
        "question": "The question text (e.g., 'Which of the following... Select all that apply')",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct_indices": [0, 2],
        "explanation": "A short explanation of why these specific answers are correct."
    }}
]

Important:
- Return ONLY the JSON code block.
- 'correct_indices' MUST be a LIST of integers (e.g., [0, 2] or [1, 2, 3]).
- Ensure that for each question, at least one (and often more than one) option is correct.
""",
    "Theoretical (answer in 30 words)": """
Generate {number_of_questions} Theoretical (answer in 30 words) questions for a {role} interview.
Level: {level}
Topic(s): '''{topic}'''

Output MUST be a raw JSON list of dictionaries with this exact structure:
[
    {{
        "question": "The question text."
     }}
]

Important:
- Return ONLY the JSON code block.
- Generate a novel variant each time; avoid repeating previous phrasing; use different examples. Feel free to include scenario-based questions or code-snippet-based questions.
""",
    "Programming (Hands-on)": """
Generate {number_of_questions} Programming (Hands-on) question for a {role} interview.
Level: {level}
Topics: '''{topic}'''

Important:
- Output MUST be a Markdown format.
- Generate a novel variant each time; use different examples.
"""
}

number_of_questions_dictionary = {
    "MCQ (Single Correct)": 5,
    "MCQ (Multiple Correct)": 3,
    "Theoretical (answer in 30 words)": 2,
    "Programming (Hands-on)": 1
}

temprature_dictionary = {
    "MCQ (Single Correct)": 1.3,    # 1
    "MCQ (Multiple Correct)": 1.3,    # 1
    "Theoretical (answer in 30 words)": 1.6,
    "Programming (Hands-on)": 1.6
}

def generate_question_prompt(inputs):
    try:
        question_type, role, level, topic = inputs['question_type'], \
                                            inputs['role'], \
                                            inputs['level'], \
                                            inputs['topic']

        prompt = prompt_dictionary[question_type]
        return prompt.format(number_of_questions = number_of_questions_dictionary[question_type],
                             role = role,
                             level = level,
                             topic = topic)
    except KeyError:
        return f"Error: Template '{question_type}' not found."
    except Exception as e:
        return f"Error formatting message: {e}"





def generate_questions(prompt: str,
                       model_name: str,
                       question_type: str):
    """
    Calls Gemini, extracts JSON from the response text (optionally wrapped in ``````),
    parses it, and returns (questions_data, raw_text).

    Raises:
        ValueError: if JSON parsing fails (includes raw response text in the error).
    """
    model = genai.GenerativeModel(model_name)
    temperature = temprature_dictionary[question_type]

    max_retries = 3
    attempt = 0

    while True:
        response = model.generate_content(
            prompt,
            generation_config={"temperature": temperature}
        )

        try:
            if question_type in ["MCQ (Single Correct)",
                                 "MCQ (Multiple Correct)",
                                 "Theoretical (answer in 30 words)"]:
                raw_text = (response.text or "").replace("``````", "").strip()
                questions_data = json.loads(raw_text)
                return questions_data

            elif question_type in ["Programming (Hands-on)"]:
                raw_text = (response.text or "").strip()
                questions_data = [{"question": raw_text.replace("``````", "").strip()}]
                return questions_data

        except json.JSONDecodeError as e:
            attempt += 1
            if attempt >= max_retries:
                raise ValueError(f"Error parsing JSON: {e}\nRaw Response: {raw_text}") from e

            time.sleep(0.15 * attempt)    # small backoff before retry





def single_correct_mcq(questions_list):
    quiz_elements = []

    # 1. Generate Widgets
    for q in questions_list:
        q_label = widgets.HTML(value=f"<b>{q['question']}</b>")

        q_options = widgets.RadioButtons(
            options=q['options'],
            index=None,
            disabled=False,
            layout={'width': 'max-content'}
        )

        q_feedback = widgets.Output()

        quiz_elements.append({
            'question_data': q,
            'radio_widget': q_options,
            'feedback_widget': q_feedback,
            'container': widgets.VBox([q_label, q_options, q_feedback, widgets.HTML("<hr>")])
        })

    # 2. Submit Button & Score
    submit_button = widgets.Button(
        description="Submit",
        button_style='primary'
    )

    final_score_output = widgets.Output()

    # 3. Grading Logic
    def on_submit_click(b):
        score = 0
        total = len(questions_list)

        for item in quiz_elements:
            user_choice_index = item['radio_widget'].index
            correct_index = item['question_data']['correct_index']
            explanation_text = item['question_data'].get('explanation', 'No explanation provided.')
            feedback_out = item['feedback_widget']

            with feedback_out:
                clear_output(wait=True)

                if user_choice_index is None:
                    print("⚠️ You didn't select an answer.")
                    # We show the explanation anyway so they can learn
                    print(f"\n📖 Explanation: {explanation_text}")

                elif user_choice_index == correct_index:
                    print("✅ Correct!")
                    print(f"\n📖 Explanation: {explanation_text}")
                    score += 1
                else:
                    correct_ans = item['question_data']['options'][correct_index]
                    print(f"❌ Incorrect. The correct answer was: {correct_ans}")
                    print(f"\n📖 Explanation: {explanation_text}")

        # Final Score
        with final_score_output:
            clear_output(wait=True)
            print(f"FINAL SCORE: {score} / {total}")

    submit_button.on_click(on_submit_click)

    # 4. Display
    question_boxes = [item['container'] for item in quiz_elements]
    display(widgets.VBox(question_boxes + [submit_button, final_score_output]))





import ipywidgets as widgets
from IPython.display import display, clear_output

def mulitple_correct_mcq(questions_list):
    """
    Renders a quiz that supports multiple correct answers using Checkboxes.
    """
    quiz_elements = []

    # 1. Generate Widgets
    for q in questions_list:
        q_label = widgets.HTML(value=f"<b>{q['question']}</b>")

        # Create a list of checkboxes, one for each option
        checkboxes = [
            widgets.Checkbox(description=opt, indent=False, layout={'width': 'max-content'})
            for opt in q['options']
        ]
        # Group checkboxes in a VBox
        options_container = widgets.VBox(checkboxes)

        q_feedback = widgets.Output()

        quiz_elements.append({
            'question_data': q,
            'checkbox_widgets': checkboxes, # Store the list of checkboxes
            'feedback_widget': q_feedback,
            'container': widgets.VBox([q_label, options_container, q_feedback, widgets.HTML("<hr>")])
        })

    # 2. Submit Button & Score
    submit_button = widgets.Button(
        description="Submit",
        button_style='primary')

    final_score_output = widgets.Output()

    # 3. Grading Logic
    def on_submit_click(b):
        score = 0
        total = len(questions_list)

        for item in quiz_elements:
            # Get indices of all checkboxes that are checked
            user_selected_indices = [
                i for i, cb in enumerate(item['checkbox_widgets']) if cb.value
            ]

            correct_indices = sorted(item['question_data']['correct_indices'])
            explanation_text = item['question_data'].get('explanation', 'No explanation.')
            feedback_out = item['feedback_widget']

            with feedback_out:
                clear_output(wait=True)

                # Check if user selection matches correct indices exactly
                if sorted(user_selected_indices) == correct_indices:
                    print("✅ Correct!")
                    score += 1
                else:
                    correct_labels = [item['question_data']['options'][i] for i in correct_indices]
                    print(f"❌ Incorrect. Correct answers: {', '.join(correct_labels)}")

                print(f"📖 Explanation: {explanation_text}")

        with final_score_output:
            clear_output(wait=True)
            print(f"FINAL SCORE: {score} / {total}")

    submit_button.on_click(on_submit_click)

    # 4. Display
    question_boxes = [item['container'] for item in quiz_elements]
    display(widgets.VBox(question_boxes + [submit_button, final_score_output]))





def theoretical_upto_30_words(questions_data):
    SYSTEM_PROMPT = """
    You are a strict, fair evaluator for short-answer questions.

    Grade each student answer for correctness and completeness relative to the question.
    Return a per-question score from 0 to 5 (integer only) and a brief explanation.

    Scoring rubric (0–5):
    - 5: Fully correct and complete; no meaningful errors.
    - 4: Mostly correct; minor omission or minor imprecision.
    - 3: Partially correct; key gaps, unclear reasoning, or some incorrect detail.
    - 2: Slight understanding; mostly incorrect/incomplete but has a relevant piece.
    - 1: Minimal understanding; largely incorrect/irrelevant.
    - 0: Blank, “I don’t know”, refusal, or totally irrelevant.

    Output requirements (must follow exactly):
    - Output ONLY valid JSON (no Markdown, no code fences, no extra text).
    - Preserve the same order as the input questions.
    - If an answer is missing/empty, score 0 with a short explanation.
    - Keep explanations concise (1–3 sentences) and actionable.
    """.strip()


    def build_user_prompt(questions, student_answers):
        payload = {
            "questions": [
                {"id": i + 1, "question": q, "student_answer": student_answers[i]}
                for i, q in enumerate(questions)
            ]
        }

        return f"""
    Evaluate the student answers for the following questions.

    INPUT_JSON:
    {json.dumps(payload, ensure_ascii=False)}

    Return JSON in exactly this format:
    {{
    "results": [
        {{
        "id": 1,
        "score": 0,
        "explanation": ""
        }}
    ]
    }}
    """.strip()


    model = genai.GenerativeModel(
        model_name=GEMINI_FLASH_LITE,    # GEMINI_3_FLASH
        system_instruction=SYSTEM_PROMPT
    )


    questions = [q["question"] for q in questions_data]
    # [{'question': "How does Python's `is` operator differ from the `==` operator when comparing objects?"},
    #  {'question': 'Explain the difference between a list and a tuple in Python regarding mutability.'}]

    # ----------------------------
    # 2) WIDGET FORM (with per-question feedback)
    # ----------------------------
    quiz_elements = []

    for q in questions_data:
        q_label = widgets.HTML(f"<b>{q['question']}</b>")

        ans = widgets.Textarea(
            value="",
            placeholder="Type your answer here...",
            layout=widgets.Layout(width="99%", height="80px")
        )

        q_feedback = widgets.Output()

        container = widgets.VBox([q_label, ans, q_feedback, widgets.HTML("<hr>")])

        quiz_elements.append({
            "question_data": q,
            "answer_widget": ans,
            "feedback_widget": q_feedback,
            "container": container
        })

    submit_btn = widgets.Button(description="Submit", button_style="primary")
    final_score_out = widgets.Output()

    ui = widgets.VBox([item["container"] for item in quiz_elements] + [submit_btn, final_score_out])
    display(ui)


    # ----------------------------
    # 3) SUBMIT HANDLER: call Gemini + render results
    # ----------------------------
    def on_submit_click(_):
        submit_btn.disabled = True  # prevents double-clicks while request runs
        try:
            # Collect student answers in the same order
            student_answers = [item["answer_widget"].value  if item["answer_widget"].value else '<Blank>'  for item in quiz_elements]

            # Clear all per-question feedback outputs
            for item in quiz_elements:
                item["feedback_widget"].clear_output(wait=True)

            with final_score_out:
                clear_output(wait=True)
                print("Evaluating...")

            user_prompt = build_user_prompt(questions, student_answers)

            response = model.generate_content(
                user_prompt,
                generation_config={"response_mime_type": "application/json"}
            )
            result_obj = json.loads(response.text)

            # Build lookup by id (robust even if order changes)
            results_by_id = {r["id"]: r for r in result_obj.get("results", [])}

            total_score = 0
            max_total = 5 * len(quiz_elements)

            # Render per-question feedback
            for idx, item in enumerate(quiz_elements, start=1):
                r = results_by_id.get(idx, {"score": 0, "explanation": "No result returned by evaluator."})
                score = int(r.get("score", 0))
                explanation = r.get("explanation", "")

                total_score += max(0, min(5, score))

                # Using Output() context manager is the most reliable way to print into it
                with item["feedback_widget"]:
                    print(f"Score: {score}/5")
                    print(f"Explanation: {explanation}")

            # Final score
            with final_score_out:
                clear_output(wait=True)
                print(f"FINAL SCORE: {total_score} / {max_total}")

        except Exception as e:
            with final_score_out:
                clear_output(wait=True)
                print("Error during evaluation:")
                print(str(e))
        finally:
            submit_btn.disabled = False


    submit_btn.on_click(on_submit_click)


# theoretical_upto_30_words(questions_data)





def programming_hands_on(questions_data):
    SYSTEM_PROMPT = """
    You are a strict, fair evaluator for hands-on programming interview questions (Python and SQL).

    Your job:
    - Evaluate each student submission for correctness, completeness, and code quality relative to the question’s requirements.
    - Prefer evidence-based grading: check whether the solution would run (Python) or execute and return the correct result set (SQL), and whether it handles important edge cases.
    - If a reference solution, expected output, test cases, or constraints are provided, use them as the primary grading source.

    What to evaluate (per question):
    1) Correctness
    - Produces the expected output for the described input(s).
    - Uses correct SQL semantics (JOIN type, filtering, grouping) or correct Python logic (data structures, loops, complexity).
    2) Edge cases and robustness
    - Handles null/empty inputs, duplicates, boundary values, and typical pitfalls relevant to the task.
    - For SQL: NULL handling, join cardinality issues, filter placement (WHERE vs HAVING), grouping correctness.
    3) Clarity and maintainability
    - Readable naming, clear structure, minimal unnecessary complexity.
    - For SQL: clear aliasing, logical query structure.
    4) Efficiency (lightweight)
    - Reasonable time/space complexity for the stated constraints.
    - Avoids obviously inefficient patterns when a simpler approach exists.

    Scoring rubric (0–5):
    - 5: Correct and complete; handles key edge cases; clean solution; efficient for constraints.
    - 4: Correct for typical cases; minor edge-case gap or minor style/efficiency issue.
    - 3: Partially correct; passes some cases but has notable logical gaps or missing key requirement.
    - 2: Some relevant approach but mostly incorrect; would fail many cases; major misunderstanding.
    - 1: Minimal progress; fragments of correct syntax/ideas but not a workable solution.
    - 0: Blank, refusal, or irrelevant.

    Output requirements (must follow exactly):
    - Output ONLY valid JSON (no Markdown, no code fences, no extra text).
    - Preserve the same order as the input questions.
    - Score must be an integer from 0 to 5.
    - Explanation must be concise (1–3 sentences) and actionable.
    - If the student answer is empty, score 0 and explain briefly.

    Extra rules:
    - Do NOT execute code; reason about it.
    - If multiple valid solutions exist, grade based on whether the answer satisfies the requirements, not on matching a specific approach.
    - If the prompt lacks enough details to verify correctness, state the key missing detail(s) and grade conservatively.
    """.strip()


    def build_user_prompt(questions, student_answers):
        payload = {
            "questions": [
                {"id": i + 1, "question": q, "student_answer": student_answers[i]}
                for i, q in enumerate(questions)
            ]
        }

        return f"""
    Evaluate the student answers for the following questions.

    INPUT_JSON:
    {json.dumps(payload, ensure_ascii=False)}

    Return JSON in exactly this format:
    {{
    "results": [
        {{
        "id": 1,
        "score": 0,
        "explanation": ""
        }}
    ]
    }}
    """.strip()


    model = genai.GenerativeModel(
        model_name=GEMINI_3_FLASH,
        system_instruction=SYSTEM_PROMPT
    )


    questions = [q["question"] for q in questions_data]
    # [{'question': "How does Python's `is` operator differ from the `==` operator when comparing objects?"},
    #  {'question': 'Explain the difference between a list and a tuple in Python regarding mutability.'}]

    # ----------------------------
    # 2) WIDGET FORM (with per-question feedback)
    # ----------------------------
    quiz_elements = []

    for q in questions_data:
        q_label = widgets.Output()

        with q_label:
            clear_output(wait=True)
            display(Markdown(questions_data[0]["question"]))


        ans = widgets.Textarea(
            value="",
            placeholder="Type your answer here...",
            layout=widgets.Layout(width="99%", height="180px")
        )

        q_feedback = widgets.Output()

        container = widgets.VBox([q_label, ans, q_feedback, widgets.HTML("<hr>")])

        quiz_elements.append({
            "question_data": q,
            "answer_widget": ans,
            "feedback_widget": q_feedback,
            "container": container
        })

    submit_btn = widgets.Button(description="Submit", button_style="primary")
    final_score_out = widgets.Output()

    ui = widgets.VBox([item["container"] for item in quiz_elements] + [submit_btn, final_score_out])
    display(ui)


    # ----------------------------
    # 3) SUBMIT HANDLER: call Gemini + render results
    # ----------------------------
    def on_submit_click(_):
        submit_btn.disabled = True  # prevents double-clicks while request runs
        try:
            # Collect student answers in the same order
            student_answers = [item["answer_widget"].value for item in quiz_elements]

            # Clear all per-question feedback outputs
            for item in quiz_elements:
                item["feedback_widget"].clear_output(wait=True)

            with final_score_out:
                clear_output(wait=True)
                print("Evaluating...")

            user_prompt = build_user_prompt(questions, student_answers)

            response = model.generate_content(
                user_prompt,
                generation_config={"response_mime_type": "application/json"}
            )
            result_obj = json.loads(response.text)

            # Build lookup by id (robust even if order changes)
            results_by_id = {r["id"]: r for r in result_obj.get("results", [])}

            total_score = 0
            max_total = 5 * len(quiz_elements)

            # Render per-question feedback
            for idx, item in enumerate(quiz_elements, start=1):
                r = results_by_id.get(idx, {"score": 0, "explanation": "No result returned by evaluator."})
                score = int(r.get("score", 0))
                explanation = r.get("explanation", "")

                total_score += max(0, min(5, score))

                # Using Output() context manager is the most reliable way to print into it
                with item["feedback_widget"]:
                    print(f"Score: {score}/5")
                    print(f"Explanation: {explanation}")

            # Final score
            with final_score_out:
                clear_output(wait=True)
                print(f"FINAL SCORE: {total_score} / {max_total}")

        except Exception as e:
            with final_score_out:
                clear_output(wait=True)
                print("Error during evaluation:")
                print(str(e))
        finally:
            submit_btn.disabled = False


    submit_btn.on_click(on_submit_click)


# programming_hands_on(questions_data)





import ipywidgets as widgets
from IPython.display import display

# --- Inputs ---
role_w = widgets.Text(
    value="Software Engineer",
    description="Role:",
    style={"description_width": "140px"},
    layout=widgets.Layout(width="600px"),
)

question_type_w = widgets.Dropdown(
    options=[
        "MCQ (Single Correct)",
        "MCQ (Multiple Correct)",
        "Theoretical (answer in 30 words)",
        "Programming (Hands-on)",
    ],
    value="MCQ (Single Correct)",
    description="Question Type:",
    style={"description_width": "140px"},
    layout=widgets.Layout(width="600px"),
)

topic_w = widgets.Textarea(
    value="SQL",
    description="Topic:",
    style={"description_width": "140px"},
    layout=widgets.Layout(width="600px", height="70px"),
)

level_w = widgets.Dropdown(
    options=["Beginner (Junior)", "Intermediate (Mid-level)", "Advanced (Senior/Expert)"],
    value="Beginner (Junior)",
    description="Level:",
    style={"description_width": "140px"},
    layout=widgets.Layout(width="600px"),
)

# --- Proceed button + output area ---
proceed_btn = widgets.Button(
    description="Proceed",
    button_style="primary",
    tooltip="Generate questions",
)

out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "8px"})

def get_inputs():
    return {
        "role": role_w.value,
        "question_type": question_type_w.value,
        "topic": topic_w.value,
        "level": level_w.value,
    }

# Wire the click event (Button.on_click) and render into an Output widget [web:23][web:24]
@out.capture(clear_output=True)
def on_proceed_clicked(_):
    inputs = get_inputs()
    # print(inputs)

    prompt = generate_question_prompt(inputs)
    # print(prompt)

    questions_data = generate_questions(prompt, GEMINI_FLASH_LITE, inputs["question_type"])
    # print(questions_data)

    if inputs['question_type']=="MCQ (Single Correct)":
        single_correct_mcq(questions_data)
    elif inputs['question_type']=="MCQ (Multiple Correct)":
        mulitple_correct_mcq(questions_data)
    elif inputs['question_type']=="Theoretical (answer in 30 words)":
        theoretical_upto_30_words(questions_data)
    elif inputs['question_type']=="Programming (Hands-on)":
        programming_hands_on(questions_data)



proceed_btn.on_click(on_proceed_clicked)  # on_click registers the handler [web:23]

# --- Layout ---
form = widgets.VBox(
    [
        role_w,
        question_type_w,
        # num_questions_w,
        topic_w,
        level_w,
        proceed_btn,
        out,
    ]
)


display(form)